In [1]:
using SparseArrays
using LinearAlgebra
using Plots
using Statistics
using SuiteSparse
using VectorizedRoutines # for meshgrid

In [2]:
function lgwt(N,a,b)
    # lgwt.m This script is for computing definite integrals using Legendre-Gauss Quadrature.Computes the Legendre - Gauss nodes and weights on an interval [a, b] with truncation order N
    # Suppose you have a continuous function f(x) which is defined on[a, b]  which you can evaluate at any x in [a, b].Simply evaluate itat all of the values contained in the x vector to obtain a vector f.Then compute the definite integral using sum(f. * w);
    # Written by Greg von Winckel - 02 / 25 / 2004
    N = N - 1
    N1 = N + 1
    N2 = N + 2
    xu = range(-1,stop =  1, length =N1)
    xu = xu'

    # Initial guess
    y = cos.((2 * (0:N)'.+1)*pi/(2*N+2)).+(0.27/N1)*sin.(pi*xu*N/N2)
    # Legendre-Gauss Vandermonde Matrix
    L=zeros(N1, N2) # vu l'algo on pourrait avoir juste 2 variables vectorielles lk-1 et lk-2 pour moins de stockage

    # Compute the zeros of the N + 1 Legendre Polynomial using the recursion relation and the Newton - Raphson method
    y0 = 2
    eps = 2.2204e-16
    Lp = 0
    # Iterate until new points are uniformly within epsilon of old points
    while maximum(abs.(y .- y0)) > eps
        L[:, 1].=1
        L[:, 2]=y
        for k = 3:N2
            L[:, k]=((2*(k-1) -1 ) * y[:] .* L[:, k-1] - (k-2) * L[:, k-2] )/ (k-1) ; 
        end
        Lp = N2 * (L[:, N1] - y[:] .* L[:, N2])./ (1 .- y[:].^2);
        y0 = y
        y = y0[:] .- L[:, N2]./ Lp[:];
    end
    # Linear map from [-1, 1] to[a, b]
    x = (a * (1 .- y) + b * (1 .+ y)) / 2
    # Compute the weights
    w = (b - a) / ((1 .- y[:].^2) .* Lp[:].^2) * (N2 / N1)^2
    return x, w 
end

lgwt (generic function with 1 method)

In [3]:
function acoscplx(x)#prolonge acos sur C
    y = complex(x)
    z = y + im*(1-y*y)^0.5
    return -im*log(z)
end

acoscplx (generic function with 1 method)

In [17]:
function Wgp(x,y,Xc,method = "MMC",alp=1, epsi =0.866, bet=1e-3)
#  Evaluate characteristic function in each Gauss point
    End = length(Xc)
    ii=1:length(x);   
    X=Xc[1:6:End];
    Y=Xc[2:6:End];
    L=Xc[3:6:End];
    h=Xc[4:6:End];
    T=Xc[5:6:End];
    jj=1:length(X);
    I,J=Matlab.meshgrid(ii,jj)
    xi=reshape(x[I],size(I));
    yi=reshape(y[I],size(I));
    rho=sqrt.((X[J]-xi).^2+(Y[J]-yi).^2);
    drho_dX=(X[J]-xi)./(rho.+(rho==0));
    drho_dY=(Y[J]-yi)./(rho.+(rho==0));
    phi=atan.(-Y[J]+yi,-(X[J]-xi))-T[J]; 
    dphi_dX=((-Y[J]+yi)./(rho.^2 .+(rho==0)));
    dphi_dY=(X[J]-xi)./(rho.^2 .+(rho==0));
    dphi_dT=-ones(size(J));
    upsi=sqrt.(rho.^2+L[J].^2/4-rho.*L[J].*abs.(cos.(phi))).*(((rho.*cos.(phi)).^2) .>=(L[J].^2/4))+ .!(((rho.*cos.(phi)).^2).>=(L[J].^2/4)).*abs.(rho.*sin.(phi));
    dupsi_drho=(2*rho-L[J].*abs.(cos.(phi)))/2 ./(upsi .+(upsi==0)).*((((rho.*cos.(phi)).^2) .>=(L[J].^2/4)))+ .~(((rho.*cos.(phi)).^2) .>=(L[J].^2/4)).*abs.(sin.(phi));
    dupsi_dphi=(L[J].*rho.*sign.(cos.(phi)).*sin.(phi))/2 ./(upsi .+(upsi==0)).*((((rho.*cos.(phi)).^2) .>=(L[J].^2/4)))+ .~(((rho.*cos.(phi)).^2) .>=(L[J].^2/4)).*rho.*sign.(sin.(phi)).*cos.(phi);
    dupsi_dL=(L[J]/2-rho.*abs.(cos.(phi)))./2 ./(upsi .+(upsi==0)).*((((rho.*cos.(phi)).^2) .>=(L[J].^2/4)) .&reshape([ i != 0 for i in upsi],size(upsi)));
    if method =="MMC"
        chi0=1 .-(4*upsi.^2 ./h[J].^2).^alp;
        dchi0_dh=8*alp*upsi.^2 .*(4*upsi.^2 ./h[J].^2).^(alp-1)./h.^3;
        dchi0_dupsi=-8*alp*upsi.*(4*upsi.^2 ./h[J].^2).^(alp-1)./h.^2;
        chi,dchi=Aggregation_Pi(chi0); 
        dchi_dh=(dchi0_dh.*dchi);
        dchi_dupsi=(dchi0_dupsi.*dchi);
        chi[chi .<=-1e6] .=-1e6;#seuil min
        W=(chi .>epsi)+((chi .<=epsi) .& (chi .>=-epsi)).*(3/4*(1-bet)*(chi/epsi .-chi.^3/3/epsi^3).+(1+bet)/2)+(chi .<-epsi)*bet;
        dW_dchi=-3/4*(1/epsi .-chi.^2/epsi^3).*(bet-1).*(abs.(chi) .<epsi);
        dW_dupsi=repeat(dW_dchi,size(dchi_dh,1)).*dchi_dupsi;
        dW_dh=repeat(dW_dchi,size(dchi_dh,1)).*dchi_dh;
        dW_dX=dW_dupsi.*(dupsi_dphi.*dphi_dX+dupsi_drho.*drho_dX);
        dW_dY=dW_dupsi.*(dupsi_dphi.*dphi_dY+dupsi_drho.*drho_dY);
        dW_dL=dW_dupsi.*dupsi_dL;
        dW_dT=dW_dupsi.*dupsi_dphi.*dphi_dT;
    elseif method == "GP" #ok ca marche celui-la
        #deltamin=p.deltamin;
        #r=p.r;
        zetavar=upsi-h[J]/2;
        dzetavar_dupsi=ones(size(upsi));
        dzetavar_dh=-0.5*ones(size(J));
        deltaiel=(1/pi/r^2*(r^2*acoscplx.(zetavar/r))-zetavar.*sqrt.(complex.(r^2 .-zetavar.^2))).*(abs.(zetavar) .<=r)+((zetavar .<-r));
        ddetlaiel_dzetavar=(-2*sqrt.(complex.(r^2 .-zetavar.^2)/pi/r^2)).*(abs.(zetavar) .<=r);
        W=deltamin .+(1-deltamin)*deltaiel;
        dW_ddeltaiel=(1-deltamin);
        dW_dh=dW_ddeltaiel*ddetlaiel_dzetavar.*dzetavar_dh;
        dW_dupsi=dW_ddeltaiel*ddetlaiel_dzetavar.*dzetavar_dupsi;
        dW_dX=dW_dupsi.*(dupsi_dphi.*dphi_dX+dupsi_drho.*drho_dX);
        dW_dY=dW_dupsi.*(dupsi_dphi.*dphi_dY+dupsi_drho.*drho_dY);
        dW_dL=dW_dupsi.*dupsi_dL;
        dW_dT=dW_dupsi.*dupsi_dphi.*dphi_dT;
    elseif method == "MNA" #ok
        #epsi=p.sigma;
        epsi = sigma
        ds=upsi;
        d=abs.(upsi);
        l=h[J]/2 .-epsi/2;
        u=h[J]/2 .+epsi/2;
        a3= -2 ./((l - u).*(l.^2 - 2*l.*u + u.^2));
        a2=   (3*(l + u))./((l - u).*(l.^2 - 2*l.*u + u.^2));
        a1=    -(6*l.*u)./((l - u).*(l.^2 - 2*l.*u + u.^2));
        a0=(u.*(- u.^2 + 3*l.*u))./((l - u).*(l.^2 - 2*l.*u + u.^2));
        W=1*(d .<= l)+(a3.*d.^3+a2 .*d.^2+a1.*d+a0).*((d .<=u) .& (d .>l));
        dW_dupsi=sign.(ds).*(3*a3.*d.^2+2*a2 .*d+a1).*((d .<=u) .&(d .>l));
        da3_du=- 2 ./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (2*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da2_du=3 ./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (3*(l + u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) + (3*(l + u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da1_du=- (6*l)./((l - u).*(l.^2 - 2*l.*u + u.^2)) - (6*l.*u)./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (6*l.*u.*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da0_du=(- u.^2 + 3*l.*u)./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (u.*(- u.^2 + 3*l.*u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) + (u.*(3*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (u.*(- u.^2 + 3*l.*u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        dWf_du=(da3_du.*d.^3+da2_du.*d.^2+da1_du.*d+da0_du).*((d.<=u).&(d.>l));
        da3_dl=2 ./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) + (2*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da2_dl= 3 ./((l - u).*(l.^2 - 2*l.*u + u.^2)) - (3*(l + u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (3*(l + u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da1_dl=    (6*l.*u)./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (6*u)./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (6*l.*u.*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da0_dl= (3*u.^2)./((l - u).*(l.^2 - 2*l.*u + u.^2)) - (u.*(- u.^2 + 3*l.*u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (u.*(- u.^2 + 3*l.*u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        dWf_dl=(da3_dl.*d.^3+da2_dl.*d.^2+da1_dl.*d+da0_dl).*((d.<=u).&(d.>l));
        dW_dh=0.5*sign.(ds).*(dWf_du+dWf_dl);
        dW_dX=dW_dupsi.*(dupsi_dphi.*dphi_dX+dupsi_drho.*drho_dX);
        dW_dY=dW_dupsi.*(dupsi_dphi.*dphi_dY+dupsi_drho.*drho_dY);
        dW_dL=dW_dupsi.*dupsi_dL;
        dW_dT=dW_dupsi.*dupsi_dphi.*dphi_dT;
    end
    return W,dW_dX,dW_dY,dW_dT,dW_dL,dW_dh
end
#deltamin= 1.0000e-06
#r=0.5
#sigma = 1
#phi=Wgp(1:20,1:20,1:12)

Wgp (generic function with 5 methods)

In [5]:
# gives m with one occur of each line, and give the indexes of the first occur
#retourne r = (a sans ligne en double) et li; r[li,:]=a
function doublons(m)
    index = []
    m2 = []
    l = length(m[1])
    for i in 1:(size(m,1))
        ligne = m[i,:]
        isin = false
        j = 1
        while j <= size(m2,1)
            if m2[j,:][1]==ligne
                isin = true
                break
            end
            j+=1
        end
        append!(index,j)
        if !isin
            push!(m2,ligne)   
        end
    end
    r = zeros(size(m2,1),size(m,2) )

    for i = 1:size(m2,1)

        r[i,:] =m2[i]
    end
    return r,index
end
e,f=doublons( [10 0 0 0 ; 0 0 0 0 ; 0 0 0 0])
e[f,:]== [10 0 0 0 ; 0 0 0 0 ; 0 0 0 0]

true

In [9]:
function Aggregation_Pi(z,aggregation ="KSl",ka = 10,zp =1)#penser a mettre en arg les P.qqchose et pour ceux qui ne sont pas dasn ttes les méthodes les mettre en =0 de base
# function that make the aggregation of the values in z and also compute sensitivities
    sz1,sz2 = size(z);
    #zm=repmat(max(z),size(z,1),1) #probleme
    zem = zeros(size(z,2))
    for i = 1:size(z,2)
        zem[i]=maximum(z[:,i])
        #zm[i,:]= [maximum(z[:,j] for j in 1:size(z,2))]
    end
    zeem =zem'
    zm = repeat(zeem,sz1) #tableau des m^colonnes du max de la colonne
    #ka=p.ka; #10
    col = ones(1,sz2)
    
    if aggregation == "asymptotic" #ok
        Wa=sum!(ones(1,sz1),z);
        dWa=ones(size(z));
    elseif aggregation == "boolean" #ok
        Wa=1 .-prod(1 .-z,dims= 1);
        dWa=repeat(prod(1 .-z.* (z .!=1), dims = 1),sz1)./(1 .-z.*(z .!=1));
    
    elseif aggregation == "p-norm"#(sum((z./exp(zm)).^ka,1)).^(1/ka) ptin ca vaut pas pareil sur matlab wtf sinon ok
        #zp=p.zp; #1
        zm=zm .+zp;
        z=z .+zp;          
        Wa=exp.(zm[1,:])'.*(sum!(col,z./exp.(zm).^ka)).^(1/ka) .-zp;
        dWa=(z./exp.(zm)).^(ka-1).*repeat((sum!(col,z./exp.(zm).^ka)).^(1/ka-1),sz1);
    elseif aggregation == "p-mean" #ok,m^prob qu 'en haut a cause des approx, on espere que c est parce que mes exemples sont des gros chiffres
        #zp=p.zp;
        zm=zm .+zp;
        z=z.+zp;
        Wa=exp.(zm[1,:])'.*(mean!(col,(z./exp.(zm)).^ka)).^(1/ka) .-zp;
        dWa=1/size(z,1)^(1/ka)*(z./exp.(zm)).^(ka-1).*repeat(sum!(col,z./exp.(zm).^ka).^(1/ka-1),sz1);
    elseif aggregation == "KS" #ok
        Wa=zm[1,:]'+1/ka*log.(sum!(col,exp.(ka*(z-zm))));
        dWa=exp.(ka*(z-zm))./repeat(sum!(col,exp.(ka*(z-zm))),sz1);
    elseif aggregation == "KSl" #ok
        e = exp.(ka*(z-zm))
        m = mean!(col,e)
        Wa=zm[1,:]' .+1/ka*log.(m);
        dWa=exp.(ka*(z-zm))./repeat(sum!(col,e),sz1);
        else #ok
        e = exp.(ka*(z-zm))
        Wa=sum!(col,z.*e)./sum!(col,e);
        dWa=((e+ka*z.*e).*repeat(sum!(col,e),sz1)-repeat(sum!(col,z.*e),sz1)*ka.*e)./repeat(sum!(col,e).^2,sz1);
        
    end
    return Wa,dWa,z,zm,sz1,ka
end
Wa,dWa,z,zm,sz1,ka = Aggregation_Pi([10 0 0 0; 0 0 0 0; 1 0 3 0],"ddd")
z

3×4 Array{Int64,2}:
 10  0  0  0
  0  0  0  0
  1  0  3  0

In [51]:
function fdelta(w,gauss_w,idgp,Ngp)#pour rendre plus lisible al boucle d'optim
    #delta=sum(reshape(W(:,idgp).*repmat(gauss_weight(:)',size(W,1),1),size(W,1),[],Ngp^2),3)...
     #   ./sum(reshape(repmat(gauss_weight(:)',size(W,1),1),size(W,1),[],Ngp^2),3);
    sz1 = size(w,1)
    agglo_gauss = repeat(gauss_w[:]', sz1)
    println("sz gauss_w ",size(gauss_w))
    println("sz ag gos ", size(agglo_gauss))
    println("sz widgp ", size(w[:,idgp]))
    println("taille ", length(w[:,idgp].*agglo_gauss))
    n = convert(Int64,length(w)/sz1/(Ngp^2))
    num = reshape(w[:,idgp].*agglo_gauss,sz1,n,Ngp^2) #size a obtenir 18x300x4
    numer = sum!(ones(1,1,Ngp^2),num)
    den = reshape(agglo_gauss, sz1,n,Ngp^2)
    denom = sum!(ones(1,1,Ngp^2),den)
    return numer ./ denom
end # a tester

fdelta (generic function with 1 method)

In [52]:
ga = ones(12,1)
w = ones(5,12)
idgp = 1:12
Ngp = 2
a = fdelta(w,ga,idgp,Ngp)


sz gauss_w (12, 1)
sz ag gos (5, 12)
sz widgp (5, 12)
taille 60


1×1×4 Array{Float64,3}:
[:, :, 1] =
 1.0

[:, :, 2] =
 1.0

[:, :, 3] =
 1.0

[:, :, 4] =
 1.0

In [55]:
function topggp(nelx,nely,settings = "GGP" , volfrac = 0.5,maxiteration = 25  )
    starting_guess = "Crosses"
    stopping_criteria="change"  #Stopping Criteria:  'change'  'kktnorm'
    if settings=="GGP"
        method="MMC"   #Method:   'MMC'  'MNA'  'GP'
        q=1                 #q=1
        zp=1           #parameter for p-norm/mean regularization
        alp=1          #parameter for MMC
        epsi=0.866     #parameter for MMC
        bet=1e-3       #parameter for MMC
        minh=1              # minimum height of a component
        aggregation="KSl"   #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10          #parameter for the aggregation constant
        saturation=true     #switch for saturation
        ncx=1           #number of components in the x direction
        ncy=1           #number of components in the y direction
        Ngp=2           #number of Gauss point per sampling window
        R=0.5          #radius of the sampling window (infty norm)
        initial_d=0.5    #component initial mass
    elseif settings=="MMC"
        method="MMC"  #'MMC'  'MNA'  'GP'
        q=1                 #q=1
        zp=1           #parameter for p-norm/mean regularization
        alp=1          #parameter for MMC
        epsi=0.866     #parameter for MMC
        bet=1e-3       #parameter for MMC
        minh=1              # minimum height of a component
        aggregation="KSl"  #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10          #parameter for the aggregation constant
        saturation=true    #switch for saturation
        ncx=1               #number of components in the x direction
        ncy=1               #number of components in the y direction
        Ngp=2               #number of Gauss point per sampling window
        R=0.5    #radius of the sampling window (infty norm)
        initial_d=0.5         #component initial mass
    elseif settings=="MNA"
        method="MNA"    #'MMC'  'MNA'  'GP'
        q=1             #q=1
        zp=1           #parameter for p-norm/mean regularization
        minh=1              # minimum height of a component
        sigma=1        #parameter for MNA
        penalty=3      #parameter for MNA
        gammav=1       #parameter for GP
        gammac=3       #parameter for GP
        aggregation="KSl"    #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10           #parameter for the aggregation constant
        saturation=true     #switch for saturation
        ncx=1               #number of components in the x direction
        ncy=1               #number of components in the y direction
        Ngp=2               #number of Gauss point per sampling window
        R=0.5               #radius of the sampling window (infty norm)
        initial_d=0.5       #component initial mass
    elseif settings=="GP"
        method="GP"    #'MMC'  'MNA'  'GP'
        q=1                 #q=1
        zp=1           #parameter for p-norm/mean regularization
        deltamin=1e-6  #parameter for GP
        r=0.5          #parameter for GP
        minh=1              # minimum height of a component
        gammav=1       #parameter for GP
        gammac=3       #parameter for GP
        aggregation="KSl"  #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10          #parameter for the aggregation constant
        saturation=true   #switch for saturation
        ncx=1               #number of components in the x direction
        ncy=1               #number of components in the y direction
        Ngp=2               #number of Gauss point per sampling window
        R=0.5    #radius of the sampling window (infty norm)
        initial_d=0.5       #component initial mass
    else
        print("settings string should be a valid entry: GGP, MMC, MNA or GP") 
        return        
    end
    E=1
    nu=0.3
    E0 = 1;
    Emin = 1e-6
    A11 = [12  3 -6 -3;  3 12  3  0; -6  3 12 -3; -3  0 -3 12]
    A12 = [-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6]
    B11 = [-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4]
    B12 = [ 2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2]
    KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11])

    nodenrs = reshape(1:(1+nelx)*(1+nely),1+nely,1+nelx); #number of the nodes in columns
    edofVec = reshape(2*nodenrs[1:end-1,1:end-1].+1,nelx*nely,1) ;#1st dof of each element (x top left)
    edofMat = zeros(nelx*nely, 8); #every line i contains the 8 dof of the ith element
    noeudsvoisins = [0 1 2*nely.+[2 3 0 1] -2 -1];
    for i = 1:8
        for j = 1:nelx*nely
            edofMat[j,i]= edofVec[j]+ noeudsvoisins[i] ;
        end
    end
    iK = reshape(kron(edofMat,ones(8,1))',64*nelx*nely,1);# line to build  K
    jK = reshape(kron(edofMat,ones(1,8))',64*nelx*nely,1);# columns
    
        # Define the nodal coordinates
    x=1
    Yy=zeros((nelx+1)*(nely+1))
    Xx=zeros((nelx+1)*(nely+1))

    for i = 1:(nelx+1)
        for j = 1:(nely+1)
            Yy[x]= j
            Xx[x]= i
            x+=1
        end
    end
    Yy=(nely+1).-Yy
    Xx=Xx.-1

        # Compute the centroid coordinates
    xc = zeros(nelx * nely)
    yc = zeros(nelx * nely)
    for xi = 0:(nelx-1)
        xc[(xi*nely+1):(xi+1)*nely].=xi+0.5
    end
    for yi = 0:(nely-1)
        yc[(yi+1):nely:nelx*nely].=yi+0.5
    end
    yc = nely .- yc
              
    centroid_coordinate=zeros(nelx*nely,2)
    for i = 1:nelx*nely
        centroid_coordinate[i,1]= xc[i]
        centroid_coordinate[i,2]= yc[i]
    end
    #l= 2*nelx*nely #length(centroid_coordinate)
    a=-R
    b=R
    gpc,wc=lgwt(Ngp,a,b)

    gpcx,gpcy=Matlab.meshgrid(gpc,gpc) #gpcx de taille ngp^2 normalement
    gauss_w = wc' * wc
    gpcx = repeat( gpcx,nelx*nely)[:] #ici on diffère de matlab...influence ? on verra
    gpcy = repeat( gpcy,nelx*nely)[:]
    gauss_weight=repeat(gauss_w,nelx*nely)[:]
    cc=repeat(centroid_coordinate,Ngp^2)
    gauss_point = cc + vcat(gpcx', gpcy')'
    ugp , idgp= doublons(gauss_point)#doublons codé ici

    #design variables
    xp=range(minimum(Xx),stop=maximum(Xx),length=ncx+2)
    yp=range(minimum(Yy),stop=maximum(Yy),length=ncy+2); 
    xx,yy=Matlab.meshgrid(xp,yp);
    Xc=repeat(xx[:],2,1);
    Yc=repeat(yy[:],2,1); 
    Lc=2*sqrt.((nelx/(ncx+2))^2+(nely/(ncy+2))^2)*ones(size(Xc)); 
    OnE = ones(convert(Int64,ceil(length(Xc)/2)))
    Tc=atan.(nely/ncy,nelx/ncx)*[OnE;-OnE];# component orientation angle tetha
    hc=2*ones(length(Xc),1); # component h
    Mc=initial_d*ones(size(Xc)); # component mass (For MNA and GP)
    Xg=vcat(Xc',Yc',Lc',hc',Tc',Mc')[:]
    
    #bounds
    Xl=minimum(Xx.-1)*ones(size(Xc));Xu=maximum(Xx.+1)*ones(size(Xc));
    Yl=minimum(Yy.-1)*ones(size(Xc));Yu=maximum(Yy.+1)*ones(size(Xc));
    Ll=0*ones(size(Xc));Lu=sqrt(nelx^2+nely^2)*ones(size(Xc));
    hl=minh*ones(size(Xc));hu=sqrt(nelx^2+nely^2)*ones(size(Xc));
    Tl=-2*pi*ones(size(Xc));Tu=2*pi*ones(size(Xc));
    Ml=0*ones(size(Xc));Mu=ones(size(Xc));
    lower_bound=vcat(Xl',Yl',Ll',hl',Tl',Ml')[:]
    upper_bound=vcat(Xu',Yu',Lu',hu',Tu',Mu')[:]
    X=(Xg-lower_bound)./(upper_bound-lower_bound);
    
    #init MMA
    loop = 0;
    m = 1;
    n = length(X);
    epsimin = 0.0000001;
    eeen    = ones(n);
    eeem    = ones(m);
    zeron   = zeros(n);
    zerom   = zeros(m);
    xval    = X[:];
    xold1   = xval;
    xold2   = xval;
    xmin    = zeron;
    xmax    = eeen;
    low     = xmin;
    upp     = xmax;
    C       = 1000*eeem;
    d       = 0*eeem;
    a0      = 1;
    a       = zerom;
    outeriter = 0;
    maxoutit  = 2000;
    kkttol  =0.001;
    changetol=0.001;
    kktnorm = kkttol+10;
    outit = 0;
    change=1;
    
    #for plots
    tt=0:0.005:(2*pi);tt=repeat(tt,length(Xc));
    cc=cos.(tt);ss=sin.(tt);
    
    if stopping_criteria == "kktnorm"
        stop_cond=outit < maxoutit && kktnorm>kkttol;
    else
        stop_cond=outit < maxoutit &&change>changetol;
    end
    
    #design loop
    #while  stop_cond #remettre après
        outit   = outit+1;
        outeriter = outeriter+1;
        #Compute the smooth characteristic functions and gradients for each component 
        # on each sampling window Gauss point (Can support GPU)
        W,dW_dX,dW_dY,dW_dT,dW_dL,dW_dh=Wgp(ugp[:,1],ugp[:,2],Xg); #l183 matlab
        #Compute local volume fractions and gradients using generalized projection
        # delta is for densities, deltac for Young modulus
        delta = fdelta(W,gauss_weight,idgp,Ngp)
        ddelta_dX = fdelta(dW_dX,gauss_weight,idgp,Ngp)#verif size dw_dx = size dw_dY
        ddelta_dY = fdelta(dW_dY,gauss_weight,idgp,Ngp)
        ddelta_dT = fdelta(dW_dT,gauss_weight,idgp,Ngp)
        ddelta_dL = fdelta(dW_dL,gauss_weight,idgp,Ngp)
        ddelta_dh = fdelta(dW_dh,gauss_weight,idgp,Ngp)
    """
        delta=sum(reshape(W(:,idgp).*repmat(gauss_weight(:)',size(W,1),1),size(W,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(W,1),1),size(W,1),[],Ngp^2),3);
    ddelta_dX=sum(reshape(dW_dX(:,idgp).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_dY=sum(reshape(dW_dY(:,idgp).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_dT=sum(reshape(dW_dT(:,idgp).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_dL=sum(reshape(dW_dL(:,idgp).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_dh=sum(reshape(dW_dh(:,idgp).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    delta_c=sum(reshape(W(:,idgp).^q.*repmat(gauss_weight(:)',size(W,1),1),size(W,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(W,1),1),size(W,1),[],Ngp^2),3);
    ddelta_c_dX=sum(reshape(q*dW_dX(:,idgp).*W(:,idgp).^(q-1).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_c_dY=sum(reshape(q*dW_dY(:,idgp).*W(:,idgp).^(q-1).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_c_dT=sum(reshape(q*dW_dT(:,idgp).*W(:,idgp).^(q-1).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_c_dL=sum(reshape(q*dW_dL(:,idgp).*W(:,idgp).^(q-1).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    ddelta_c_dh=sum(reshape(q*dW_dh(:,idgp).*W(:,idgp).^(q-1).*repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3)...
        ./sum(reshape(repmat(gauss_weight(:)',size(dW_dX,1),1),size(dW_dX,1),[],Ngp^2),3);
    #end
    """
    return ddeta_dh
end
aa = topggp(20,15)

(1200,)
sz gauss_w (1200,)
sz ag gos (1, 1200)
sz widgp (1, 1200)
taille 1200


DimensionMismatch: DimensionMismatch("new dimensions (1, 150, 4) must be consistent with array size 1200")

In [63]:
function model_updateM(delta,p,X)
#update the Young Modulus on the base of delta
    m=X[6:6:length(X)];
    nc=length(m);
    m=repmat(m[:],1,size(delta,2));#verif ca
    
    if method == "MMC"
        E=E0*delta;
        dE_ddelta=E0*ones(size(delta));
        dE_ddelta=repmat(dE_ddelta,size(m,1),1);
        dE_dm=0*m;
        
    elseif method == "GP"
        hatdelta=delta.*m.^gammac;
        [E,dE_dhatdelta]=Aggregation_Pi(hatdelta,p);
        if saturation
            [E,ds]=smooth_sat(E,p,nc);
            dE_dhatdelta=ds.*dE_dhatdelta;
        end
        E=E.*E0;
        dhatdelta_ddelta=m.^gammac;
        dhatdelta_dm=gammac*delta.*m.^(gammac-1);
        dE_ddelta=E0*dhatdelta_ddelta.*dE_dhatdelta;
        dE_dm=E0*dE_dhatdelta.*dhatdelta_dm;
        
        else #MNA
        hatdelta=delta.*m.^gammac;
        [rho,drho_dhatdelta]=Aggregation_Pi(hatdelta,p);
        if saturation
            [rho,ds]=smooth_sat(rho,p,nc);
            drho_dhatdelta=ds.*drho_dhatdelta;
        end
        E=rho.^penalty.*(E0-Emin)+Emin;
        dhatdelta_ddelta=m.^gammac;
        dhatdelta_dm=gammac*delta.*m.^(gammac-1);
        dE_ddelta=penalty*(E0-Emin)*dhatdelta_ddelta.*drho_dhatdelta.*rho.^(penalty-1);
        dE_dm=penalty*(E0-Emin)*dhatdelta_dm.*drho_dhatdelta.*rho.^(penalty-1);
    end
    return E,dE_ddelta,dE_dm
end

ErrorException: syntax: invalid assignment location "[E, dE_dhatdelta]"

In [86]:
e[2,:]==e[3,:]

true

In [105]:
x= [1 3 8]
y =zeros(3,3)
for i = 1:3
    y[i,:]=x
end
z = [i for i in y[:]]
e = [7 8 9]
m = hcat(x',e',e')


1×3 Array{Int64,2}:
 1  3  8